In [ ]:
### Author : Karssien Hero Huisman
### Hartree Fock Approximation : Semi Infinite Leads
### Onsite energy of the leads shifts with voltage:
### epsilon0L,epsilon0R -> ef + V/2, ef - V/2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Import path 

In [ ]:
import sys
sys.path.insert(0, '/Users/khhuisman/Documents/Jupyter_notebooks/Paper3_SIFLeads/Modules_SIF')


### Scattering Region

In [ ]:
import Sgeom_scatteringregion #Hamiltonian without interactions U=0, not coupled to leads

### Import NEGF methods

In [ ]:
import negf_SIF_HIA as negf_method               ### negf code
import Integration_SIF_HIA_OnsiteVoltDep as Integration_method ### for calculating electron densities
import Currents_HIA_SIF as Current_method        ### for calculating currents
import Pvalue
import handy_functions_coulomb as hfc #some "usefull functions"


In [ ]:
def func_energies_voltagedepleads(Hamiltonian0,U,npoints,ef,Vmax,
                                 tleadL1,tleadR1,pz):
    

    #If tleadL or tleadR is enormous treat it like WBL
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    evlist_shifted = np.add(evlist,[U/2 for i in range(len(evlist))])
    e_lumo = evlist_shifted[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist_shifted[int(Hamiltonian0.shape[0]/2)]
    
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist_shifted))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist_shifted))/10 + 10,2)   #lower bound for integrals
    
    epsilon0_list = [ef + Vmax/2,ef - Vmax/2]
    energieslist = []
    #####
    de = 0.1
    emin_list = []
    emax_list = []
    
    for epsilon0L in epsilon0_list:
        for epsilon0R in epsilon0_list:
            
            
            
            eminL = epsilon0L - pz*2*tleadL1  -2*tleadL1 - de  
            emaxL = epsilon0L + pz*2*tleadL1 + 2*tleadL1 + de  

            eminR = epsilon0R - 2*tleadR1  - de 
            emaxR = epsilon0R + 2*tleadR1  + de  
    
    
            energieslist.append(eminL)
            energieslist.append(emaxL)
            energieslist.append(eminR)
            energieslist.append(emaxR)
    
  
    energies = np.linspace(min(energieslist),max(energieslist),npoints)
    
    return energies



In [ ]:
def plot_occupation_difference(nP_list_conv,nM_list_conv,V_list_convg,Hamiltonian0):
    dimlist = hfc.func_list_i(Hamiltonian0)
    plt.title('$n_{is}(m,V) - n_{i\overline{s}}(-m,V)$')
    n_list_total_convgM_swap = [hfc.pairwise_swap([ nM_list_conv[i][k] for k in dimlist]) for i in range(len(V_list_convg))
                              ]
    nP_list_plot =[ [nP_list_conv[i][k] for k in dimlist ] for i in range(len(V_list_convg)) ]


    plt.plot(V_list_convg,np.subtract(nP_list_plot,n_list_total_convgM_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()


def plot_densities_symmtry_relation(dim,V_list,n_list):
    ones = [1 for i in range(dim)]
    n_list_swap = [np.subtract(ones,hfc.pairwise_swap(n_list[-1-i])) for i in range(len(n_list))]

    plt.title('$n_{is}(m,V) + n_{i\overline{s}}(m,-V) - 1$')
    plt.plot(V_list,np.subtract(n_list,n_list_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))


    plt.show()
    
    
def plot_densities_symmtry_relation2(nP_list,nM_list,V_list,dim):
    
    dimlist = [i for i in range(dim)]
    ones = [1 for i in range(dim)]
    n_list_swapM = [np.subtract(ones,hfc.pairwise_swap(nM_list[-1-i])) for i in range(len(nM_list))]
    nP_list_plot =[ [nP_list[i][k] for k in dimlist ] for i in range(len(V_list)) ]

    plt.title('$n_{is}(m,V) + n_{i\overline{s}}(-m,-V) - 1$')
    plt.plot(V_list,np.subtract(nP_list_plot,n_list_swapM))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

### Constructing Scattering Region & Leads

In [ ]:
#Geometric Paramters
Lm      = 2
Wg      = 2

epsilon = 0       # onsite energy
t       = 2.4     # hopping paramter

Ulist  = np.round([2*t] ,2)
lambdalist   = [(1*(10**-1))*t]

# Temperature of leads
T = 300
betaL,betaR = negf_method.func_beta(T),negf_method.func_beta(T)



In [ ]:

tsom   = lambdalist[0] #spin-orbit coupling paramter
U      = Ulist[0]      # Coulomb interaction strength


### Hamiltonian of molecule for U = 0.
Hamiltonian0,matL,matR = Sgeom_scatteringregion.hamiltonian_multiplesites_coupled_semi_inf(Lm,Wg, t,tsom,plotbool=True)

### Parameters for Lead self-energy
tlead               = (3)*t             # hopping paramter lead strength
tleadL,tleadR       = tlead,tlead       # hopping parameter left,right lead
pz                  = 0.8               # magnetic polarization


#Coupling lead-molecule
tcoup          = np.sqrt(tlead)*0.5  #coupling of lead to the molecule
tcoupR, tcoupL = tcoup,tcoup            #coupling of right lead to the molecule


### Bias window

In [ ]:
Vmax = 1 # Maximum bias voltage [eV]
dV = 0.5 # stepsize
V_list_pos_bias,V_list_total = hfc.func_V_total(Vmax,dV)
print(len(V_list_total),V_list_pos_bias)

#### Integration & Self-consistency parameters

In [ ]:
tol             = 10**-4  # tolerance of for converence of electron densities
tol_nintegrand  = 10**-15 # cut-off value that much be satisfied for lower,upper bound Glesser integral.

max_iteration   = 100     # max number of iterations
npoints         = 30000    # max number of points to integrate over
alpha           = 0.8     # linear mixing paramter
npoints_current = 15000
ef              = 0      # fermi level 



In [ ]:
energies_self = func_energies_voltagedepleads(Hamiltonian0,U,1000,ef,Vmax,
                                 tleadL,tleadR,pz)

In [ ]:
negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,-Vmax,tleadL,tcoupL,pz)
negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,0,tleadL,tcoupL,pz)
negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,Vmax,tleadL,tcoupL,pz)

negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,-Vmax,tleadR,tcoupR,0)
negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,0,tleadL,tcoupL,0)
negf_method.plot_selfenergies_voltage_one_magnetization(energies_self,ef,Vmax,tleadR,tcoupR,0)

In [ ]:
###### DOS for exactly half-filled system 
negf_method.plot_dos(energies_self,Hamiltonian0,U,hfc.list_halves(Hamiltonian0),
                ef,tleadL,tcoupL,matL,
                ef,tleadR,tcoupR,matR,
                    pz,
                ef, ef,
                betaL,betaR)

# 3. Electron Densities

In [ ]:
factor1 = 1
factor2 = 1

In [ ]:
parameterlist = []

for U in Ulist:

    #Coupling lead-molecule
    tlead          = (3)*t             # hopping paramter lead strength
    tleadL,tleadR  = np.round(factor1*tlead,2),np.round(tlead,2)       # hopping parameter left,right lead
    tcoupL,tcoupR  =  factor2*np.sqrt(tleadL/factor1)*0.5 ,np.sqrt(tleadR)*0.5 #coupling of right lead to the molecule
    print('tleadL, tleadR = {},{} & tcoupLR = {},{}'.format(tleadL,tleadR,tcoupL,tcoupR))

    #### chi1,chi2
    chi1 = np.round(hfc.func_chi(tleadL , tleadR ),2)
    chi2 = np.round(hfc.func_chi(tcoupL , tcoupR ),2) 
    print(chi1,chi2)




    ### Scattering region
    Hamiltonian0,matL,matR = Sgeom_scatteringregion.hamiltonian_multiplesites_coupled_semi_inf(Lm,Wg, t,tsom,plotbool=False)


    ### Energies to integrate over
    energies = func_energies_voltagedepleads(Hamiltonian0,U,npoints,ef,Vmax,tleadL,tleadR,pz) #energies to integrate over.

    emin = np.round(min(energies),1)
    emax = np.round(max(energies),1)

    print(emin,emax)
    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,-Vmax,tleadL,tcoupL,pz)
    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,0,tleadL,tcoupL,pz)
    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,Vmax,tleadL,tcoupL,pz)

    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,-Vmax,tleadR,tcoupR,0)
    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,0,tleadL,tcoupL,0)
    negf_method.plot_selfenergies_voltage_one_magnetization(energies,ef,Vmax,tleadR,tcoupR,0)




    print('--- ef = {}, tsom/t = {} , U/t = {},'.format(ef,tsom/t,U/t))
    n_list_totalP,convglistP = Integration_method.self_consistent_trapz_PN_VD_energyshift(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                Hamiltonian0,U,
                                tleadL,tcoupL,matL,
                                tleadR,tcoupR,matR,
                                   abs(pz), 
                                betaL, betaR,tol,
                                tol_nintegrand,alpha,npoints,plotbool=False,trackbool=True)


    n_list_totalM,convglistM = Integration_method.self_consistent_trapz_PN_VD_energyshift(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                Hamiltonian0,U,
                                tleadL,tcoupL,matL,
                                tleadR,tcoupR,matR,
                                   -abs(pz), 
                                betaL, betaR,tol,
                                tol_nintegrand,alpha,npoints,plotbool=False,trackbool=False)








    V_list_convg,nP_list_convg,nM_list_convg =  hfc.func_symmetric_converged(V_list_total,
                                                                  n_list_totalP ,convglistP,
                                                                  n_list_totalM, convglistM) 


    parameterlist.append([npoints_current, Lm,Wg,pz,ef,ef,tleadL,tleadR,tcoupL,tcoupR, T,alpha,t,U,tsom,ef,tol,Vmax,len(V_list_total),emin,emax,npoints,tol_nintegrand])




    plot_densities_symmtry_relation(Hamiltonian0.shape[0],V_list_convg,nP_list_convg)
    plot_densities_symmtry_relation(Hamiltonian0.shape[0],V_list_convg,nM_list_convg)
    plot_densities_symmtry_relation2(nP_list_convg,nM_list_convg,V_list_convg,Hamiltonian0.shape[0])



    print('calculating currents ...')

    IP_list = Current_method.calc_I_trapz_VD(npoints_current,
                    V_list_convg,ef,
                  Hamiltonian0,
                  tleadL,tcoupL,matL,
                  tleadR,tcoupR,matR,
                  abs(pz),
                  U,nP_list_convg,
                  betaL,betaR)
    print('+M done, now -M....')
    IM_list = Current_method.calc_I_trapz_VD(npoints_current,
                    V_list_convg,ef,
                  Hamiltonian0,
                  tleadL,tcoupL,matL,
                  tleadR,tcoupR,matR,
                  -abs(pz),
                  U,nM_list_convg,
                  betaL,betaR)


    dIlist = np.subtract(IP_list,IM_list)
    Vprime, PClist = Current_method.func_MR_list(IP_list,IM_list,V_list_convg)

    plt.title('$U/t = {}$, $\lambda /t = {}$'.format(U/t,tsom/t))
    plt.plot(V_list_convg,IP_list)
    plt.plot(V_list_convg,IM_list)
    plt.xlabel('Bias Voltage [eV]')
    plt.ylabel('Current [eV]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))
    plt.show()

    plt.title('Colinear $\Delta I(m): ef = {},U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,tsom/t))
    plt.plot(V_list_convg,dIlist)
    plt.xlabel('Bias Voltage [eV]')
    plt.ylabel('Current [eV]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

    plt.title('P-Value : $E_F = {} $, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,tsom/t))
    Vlist_prime,PJ_list = Pvalue.function_PvaluedI(V_list_convg,dIlist,22)
    plt.plot(Vlist_prime,PJ_list)
    plt.xlim(0,Vmax)
    plt.ylim(-1-0.1,1+0.1)
    plt.show()



In [ ]:
print(parameterlist)